In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col  # Импортируем col()

spark = SparkSession.builder.appName('test').getOrCreate()

1. Завантажте та прочитайте кожен CSV-файл як окремий DataFrame.

In [40]:
users_df = spark.read.format("csv").option("header", "true").load("users.csv")
purchases_df = spark.read.format("csv").option("header", "true").load("purchases.csv")
products_df = spark.read.format("csv").option("header", "true").load("products.csv")

In [21]:
users_df.show(5)

+-------+------+---+-----------------+
|user_id|  name|age|            email|
+-------+------+---+-----------------+
|      1|User_1| 45|user1@example.com|
|      2|User_2| 48|user2@example.com|
|      3|User_3| 36|user3@example.com|
|      4|User_4| 46|user4@example.com|
|      5|User_5| 29|user5@example.com|
+-------+------+---+-----------------+
only showing top 5 rows



In [22]:
purchases_df.show(5)

+-----------+-------+----------+----------+--------+
|purchase_id|user_id|product_id|      date|quantity|
+-----------+-------+----------+----------+--------+
|          1|     52|         9|2022-01-01|       1|
|          2|     93|        37|2022-01-02|       8|
|          3|     15|        33|2022-01-03|       1|
|          4|     72|        42|2022-01-04|       9|
|          5|     61|        44|2022-01-05|       6|
+-----------+-------+----------+----------+--------+
only showing top 5 rows



In [23]:
products_df.show(5)

+----------+------------+-----------+-----+
|product_id|product_name|   category|price|
+----------+------------+-----------+-----+
|         1|   Product_1|     Beauty|  8.3|
|         2|   Product_2|       Home|  8.3|
|         3|   Product_3|Electronics|  9.2|
|         4|   Product_4|Electronics|  2.6|
|         5|   Product_5|Electronics|  9.4|
+----------+------------+-----------+-----+
only showing top 5 rows



2. Очистіть дані, видаляючи будь-які рядки з пропущеними значеннями.

In [46]:
print("В users_df рядків з пропусками:", users_df.count() - users_df.dropna().count())
print("В purchases_df рядків з пропусками:", purchases_df.count() - purchases_df.dropna().count())
print("В products_df рядків з пропусками:", products_df.count() - products_df.dropna().count())

В users_df рядків з пропусками: 5
В purchases_df рядків з пропусками: 5
В products_df рядків з пропусками: 3


In [47]:
users_df = users_df.dropna()
purchases_df = purchases_df.dropna()
products_df = products_df.dropna()
users_df.show(5)
purchases_df.show(5)
products_df.show(5)

+-------+------+---+-----------------+
|user_id|  name|age|            email|
+-------+------+---+-----------------+
|      1|User_1| 45|user1@example.com|
|      2|User_2| 48|user2@example.com|
|      3|User_3| 36|user3@example.com|
|      4|User_4| 46|user4@example.com|
|      5|User_5| 29|user5@example.com|
+-------+------+---+-----------------+
only showing top 5 rows

+-----------+-------+----------+----------+--------+
|purchase_id|user_id|product_id|      date|quantity|
+-----------+-------+----------+----------+--------+
|          1|     52|         9|2022-01-01|       1|
|          2|     93|        37|2022-01-02|       8|
|          3|     15|        33|2022-01-03|       1|
|          4|     72|        42|2022-01-04|       9|
|          5|     61|        44|2022-01-05|       6|
+-----------+-------+----------+----------+--------+
only showing top 5 rows

+----------+------------+-----------+-----+
|product_id|product_name|   category|price|
+----------+------------+---------

In [48]:
print("В users_df рядків з пропусками:", users_df.count() - users_df.dropna().count())
print("В purchases_df рядків з пропусками:", purchases_df.count() - purchases_df.dropna().count())
print("В products_df рядків з пропусками:", products_df.count() - products_df.dropna().count())

В users_df рядків з пропусками: 0
В purchases_df рядків з пропусками: 0
В products_df рядків з пропусками: 0


In [49]:
from pyspark.sql.functions import col, sum

users_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in users_df.columns]).show()

+-------+----+---+-----+
|user_id|name|age|email|
+-------+----+---+-----+
|      0|   0|  0|    0|
+-------+----+---+-----+



3. Визначте загальну суму покупок за кожною категорією продуктів.

In [71]:
from pyspark.sql.functions import col, sum, round
purchases_cat = purchases_df.join(products_df, on="product_id", how="inner").groupBy("category").agg(round(sum(col("price") * col("quantity")), 2).alias("total_sales"))
purchases_cat.show()

+-----------+-----------+
|   category|total_sales|
+-----------+-----------+
|       Home|     1523.5|
|     Sports|     1802.5|
|Electronics|     1174.8|
|   Clothing|      790.3|
|     Beauty|      459.9|
+-----------+-----------+



4. Визначте суму покупок за кожною категорією продуктів для вікової категорії від 18 до 25 включно.

In [93]:
age_18_25 = purchases_df.join(products_df, on="product_id", how="inner")\
    .join(users_df, on="user_id", how="inner").filter((col("age") >= 18) & (col("age") <= 25))\
    .groupBy("category").agg(round(sum(col("price") * col("quantity")), 2).alias("total_sales"))
age_18_25.show()

+-----------+-----------+
|   category|total_sales|
+-----------+-----------+
|       Home|      361.1|
|     Sports|      310.5|
|Electronics|      249.6|
|   Clothing|      245.0|
|     Beauty|       41.4|
+-----------+-----------+



5. Визначте частку покупок за кожною категорією товарів від сумарних витрат для вікової категорії від 18 до 25 років.



In [112]:
from pyspark.sql.window import Window
cat_sales = purchases_df.join(products_df, on="product_id", how="inner")\
    .join(users_df, on="user_id", how="inner").filter((col("age") >= 18) & (col("age") <= 25))\
    .groupBy("category").agg(round(sum(col("price") * col("quantity")), 2).alias("total_sales"))
total_sales = cat_sales.agg(round(sum("total_sales"), 2).alias("total_sales")).collect()[0]["total_sales"]
cat_share = cat_sales.withColumn("share", round(col("total_sales") / total_sales, 2))
cat_share.show()

+-----------+-----------+-----+
|   category|total_sales|share|
+-----------+-----------+-----+
|       Home|      361.1|  0.3|
|     Sports|      310.5| 0.26|
|Electronics|      249.6| 0.21|
|   Clothing|      245.0|  0.2|
|     Beauty|       41.4| 0.03|
+-----------+-----------+-----+



... варіант через Window-функцію

In [109]:
from pyspark.sql.window import Window
cat_share_18_25 = purchases_df.join(products_df, on="product_id", how="inner")\
    .join(users_df, on="user_id", how="inner").filter((col("age") >= 18) & (col("age") <= 25))\
    .groupBy("category").agg(round(sum(col("price") * col("quantity")), 2).alias("total_sales"))\
    .withColumn("share", round(col("total_sales") / sum("total_sales").over(Window.partitionBy()), 2))
cat_share_18_25.show()

+-----------+-----------+-----+
|   category|total_sales|share|
+-----------+-----------+-----+
|       Home|      361.1|  0.3|
|     Sports|      310.5| 0.26|
|Electronics|      249.6| 0.21|
|   Clothing|      245.0|  0.2|
|     Beauty|       41.4| 0.03|
+-----------+-----------+-----+



6. Виберіть 3 категорії продуктів з найвищим відсотком витрат споживачами віком від 18 до 25 років.

In [114]:
from pyspark.sql.window import Window
cat_sales = purchases_df.join(products_df, on="product_id", how="inner")\
    .join(users_df, on="user_id", how="inner").filter((col("age") >= 18) & (col("age") <= 25))\
    .groupBy("category").agg(round(sum(col("price") * col("quantity")), 2).alias("total_sales"))
total_sales = cat_sales.agg(round(sum("total_sales"), 2).alias("total_sales")).collect()[0]["total_sales"]
cat_share = cat_sales.withColumn("share", round(col("total_sales") / total_sales, 2)).sort(col("share").desc()).limit(3)
cat_share.show()

+-----------+-----------+-----+
|   category|total_sales|share|
+-----------+-----------+-----+
|       Home|      361.1|  0.3|
|     Sports|      310.5| 0.26|
|Electronics|      249.6| 0.21|
+-----------+-----------+-----+



In [18]:
spark.stop()